In [ ]:
import os
import re
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup


def disintegrate_file( file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Use regular expression to find all <FILENAME> tags and extract the filenames
    filenames = re.findall(r'<FILENAME>(.*?)\n', content)

    # If no <FILENAME> tags are found, generate a filename using the original filename
    if not filenames:
        base_filename = os.path.basename(file_path).replace('.txt', '') + '_document'
        filenames = [base_filename]

    # Use regular expression to find and split based on start and end tags
    documents = re.findall(r'<DOCUMENT>(.*?)</DOCUMENT>', content, re.DOTALL)
    
    # Save each document to a separate file
    for filename, doc in zip(filenames, documents):        
        processed_text = doc.strip()  

        # Save the document to a new file
        output_file_path = os.path.join(os.path.dirname(file_path), f"{filename}")
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(processed_text)


def download_10k_filings(ticker, start_year, end_year):
    dl = Downloader("MyCompanyName", "my.email@domain.com")

    for year in range(start_year, end_year + 1):
        dl.get("10-K", ticker, after=str(year-1)+"-12-31", before=str(year+1)+"-01-01",download_details=True)

        # Clean the downloaded files
        filing_path = os.path.join("sec-edgar-filings", ticker, "10-K")
        
        for root, dirs, files in os.walk(filing_path):
            for file in files:
                if file.endswith(".txt"):
                      disintegrate_file(os.path.join(root, file))

# Example usage:
companies = ["AAPL"]  # Add more tickers if needed
start_year = 2021
end_year = 2022

for company in companies:
    download_10k_filings(company, start_year, end_year)